# Remise TP : Reconnaissance des formes, méthodes structurelles
# Application : Interprétation des gestes de la main
# Travail réalisé par : Mohamed Ameziane TOUIL et Abdeslam DJEMAI 

In [1]:
# imoprtation des bibliothèques
import cv2
import numpy as np
import math

In [ ]:
#Lancement de la capture vidéo
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    # Chargement de l'image capturée
    ret, img = cap.read()

    # obtenir des données sur les mains à partir de la sous-fenêtre rectangulaire à l'écran 
    cv2.rectangle(img, (400,400), (100,100), (0,255,0),0)
    # Définition de la zone d'interêt contenant la main et les doigts
    crop_img = img[100:400, 100:400]

    # convertion en niveau de gris
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    # application du bruit gaussien
    value = (35, 35)
    blurred = cv2.GaussianBlur(grey, value, 0)

    # Seuillage: méthode d'Otsu's
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


    # vérifier la version d'OpenCV pour éviter toute erreur 
    # Récupération des contours
    (version, _, _) = cv2.__version__.split('.')

    if version == '3':
        image, contours, hierarchy = cv2.findContours(thresh1.copy(), \
               cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    elif version == '2':
        contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
               cv2.CHAIN_APPROX_NONE)

    # Ici on récupère le contours maximal dans la surface de capture sous un arriére plan Blanc
    cnt = max(contours, key = lambda x: cv2.contourArea(x))

    # Récupération du Convex Hull (enveloppe convexe)
    hull = cv2.convexHull(cnt)

    # tracage (ou dessin) des contours 
    drawing = np.zeros(crop_img.shape,np.uint8)
    # Contours de la main et des doigts
    cv2.drawContours(drawing, [cnt], 0, (0, 255, 0), 0)
    # contours de l'enveloppe Convexe
    cv2.drawContours(drawing, [hull], 0,(0, 0, 255), 0)

    # Maitenant nous recherchons encore le convex hull mais cette fois pour retrouver les defects
    hull = cv2.convexHull(cnt, returnPoints=False)

    # Récupération des convexity defects (défauts de convexité)
    defects = cv2.convexityDefects(cnt, hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0, 255, 0), 3)

    finger = False # variable pour detecter la présence d'un doigt
    
    # appliquer la règle du cosinus pour trouver l'angle pour tous les defects (entre les doigts)
    # si un angle > 90 degrés on ingnore les deffects
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        
        if d>10000: # Cette condition pour traiter le cas de zeros doigts
                        #on vérifie la distance entre le convex hull et le point qui forme l'angle
            finger = True # si la distance est supérieur à 10000 donc y'a présence d'un doigt, 10000 est trouvée
                            # après plusieurs essaies

        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])

        # trouver la longueur de tous les côtés du triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

        # appliquer la règle du cosinus ici
        angle = np.rad2deg(math.acos((b**2 + c**2 - a**2)/(2*b*c)))

        # ignorer les angles > 90 et surligner le reste avec des points rouges
        if angle <= 90:
            count_defects += 1
            cv2.circle(crop_img, far, 5, [255,0,0], -1)

        # tracer une ligne du début à la fin, c'est-à-dire les points convexes (bouts des doigts)
        cv2.line(crop_img,start, end, [0,255,0], 2)
        # Affichage des points de l'enveloppe convexe
        cv2.circle(crop_img,start,5,[0,0,255],-1)
        cv2.circle(crop_img,end,5,[0,0,255],-1)

    # Ici on définit les actions souhaitées
    if (count_defects == 0 and finger == True): # ici si y'a pas d'angle < 90 et existe un doigt donc on affiche 1
        cv2.putText(img,"Numero 1", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 1:
        str = "Numero 2"
        cv2.putText(img, str, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 2:
        cv2.putText(img,"Numero 3", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 3:
        cv2.putText(img,"Numero 4", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 4:
        cv2.putText(img,"Numero 5", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    else:
        cv2.putText(img,"Montrez moi un signe", (200, 200),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, 2)

    # Affichages des images appropriées dans des fenêtres 
    cv2.imshow('Traduction de Signe en Nombres', img)
    threshc = cv2.cvtColor(thresh1,cv2.COLOR_GRAY2BGR)
    all_img = np.hstack((threshc,drawing))
    cv2.imshow('Seuillage / Contours', all_img)

    k = cv2.waitKey(10)
    if k == 27:
        break